In [1]:
import sys
from pathlib import Path

import pandas as pd
import numpy as np
from folktables import ACSDataSource, ACSIncome

In [2]:
#root_dir = Path("/home/andre.cruz/Documents/fair-boosting/")
root_dir = Path("/mnt/home/andre.cruz/fair-boosting/")
data_dir = root_dir / "data" / "Adult-2021"
data_dir.mkdir(exist_ok=True)

# download 2018 data for 50 states
state_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
              'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
              'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
              'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
              'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

data_source = ACSDataSource(
    survey_year='2018', horizon='1-Year', survey='person',
    root_dir=str(data_dir),
)

In [3]:
STATE_COL = "ST"  # train/test splits will be stratified over states
#SENSITIVE_COL = "RAC1P"  # default
SENSITIVE_COL = "SEX"

In [4]:
# data is 3207990 rows x 286 columns
data = data_source.get_data(states=state_list, download=False)

In [5]:
from copy import deepcopy
dataset_details = deepcopy(ACSIncome)  # aka, 2021 Adult

# Add State to the feature columns so we can do stratified splits (will be removed later)
dataset_details.features.append(STATE_COL)

# Pre-process + select Adult dataset features
features, labels, groups = dataset_details.df_to_numpy(data)
df = pd.DataFrame(data=features, columns=dataset_details.features)

features.shape  # (1655429, 10) +1 for the state column

(1655429, 11)

## Order columns correctly

In [6]:
df[dataset_details.target] = labels

if SENSITIVE_COL != dataset_details.group:
    print(f"Dataset lists sensitive column as '{dataset_details.group}' "
          f"but we're using '{SENSITIVE_COL}'", file=sys.stderr)

# Correct column ordering
cols_order = [dataset_details.target, SENSITIVE_COL] + \
    list(set(dataset_details.features) - {SENSITIVE_COL, STATE_COL})

state_col = df[STATE_COL]
df = df[cols_order]

Dataset lists sensitive column as 'RAC1P' but we're using 'SEX'


In [7]:
categorical_columns = [
    "COW", # class of worker
    #"SCHL", # educational attainment (categorical? there is a clear sequence between values...)
    "MAR", # marital status
    "OCCP", # occupation
    "POBP", # place of birth
    "RELP", # relationship
    "RAC1P",
]

# Maintain column ordering
ordered_cat_columns = [col for col in df.columns if col in set(categorical_columns)]

types_dict = {col: int for col in categorical_columns}
types_dict.update({
    dataset_details.target: int,
    "SEX": int,
})

# Set categorical columns to integers
df = df.astype(types_dict)

# Set sensitive column starting at value=0
df[SENSITIVE_COL] = df[SENSITIVE_COL] - df[SENSITIVE_COL].min()

## Split Train/Test (Validation?)
* According to the Datasheet, train/test splits are performed **stratified per state**:
  * states are split **80%/20%**, and resulting data is aggregated in US-wide train/test datasets;

In [8]:
SEED = 42
TEST_PCT = 20
TRAIN_PCT = 60

VAL_PCT = 100 - TRAIN_PCT - TEST_PCT
assert VAL_PCT >= 0

In [9]:
from sklearn.model_selection import train_test_split

train_idx, other_idx = train_test_split(
    df.index,
    train_size=0.01 * TRAIN_PCT,
    stratify=state_col,
    random_state=SEED, shuffle=True)

train_df, other_df = df.loc[train_idx], df.loc[other_idx]
assert len(set(train_idx) & set(other_idx)) == 0

# Split validation
if VAL_PCT > 0:
    new_test_pct = TEST_PCT / (TEST_PCT + VAL_PCT)

    val_idx, test_idx = train_test_split(
        other_df.index,
        test_size=new_test_pct,
        stratify=state_col.loc[other_idx],
        random_state=SEED, shuffle=True)

    val_df, test_df = other_df.loc[val_idx], other_df.loc[test_idx]
    assert len(train_idx) + len(val_idx) + len(test_idx) == len(df)

assert np.isclose(len(train_df) / len(df), 0.01 * TRAIN_PCT)
assert np.isclose(len(test_df) / len(df), 0.01 * TEST_PCT)

___
___
# One-hot encoding section

### 1. Join all values that represent less than 1% of the dataset

In [10]:
def reduce_number_of_categories(df, rel_threshold=0.01, fill_value=-1, set_start_at_zero=False):
    out_df = df.copy()

    for col in df.columns:
        if col in categorical_columns:
            if out_df[col].nunique() <= 2: continue
            # Set value count starting at zero
            if set_start_at_zero:
                out_df[col] = out_df[col] - out_df[col].min()

            # Reduce the number of categories to only those that appear more than (rel_threshold * 100)%
            counts = out_df[col].value_counts(normalize=True)
            cats_above_threshold = set(counts[counts > rel_threshold].index)

            out_df[col] = out_df[col].map(lambda v: v if v in cats_above_threshold else fill_value)

    return out_df

### 1.1. Or just one-hot encode *everything*!

In [11]:
MERGE_LOW_PREVALENCE_VALUES = False

# Join feature values with low dataset prevalence
if MERGE_LOW_PREVALENCE_VALUES:
    df_preprocessed = reduce_number_of_categories(df, rel_threshold=0.01, set_start_at_zero=True)

# Or simply let everything as is :)
else:
    df_preprocessed = reduce_number_of_categories(df, rel_threshold=0.0001, set_start_at_zero=True)
    #df_preprocessed = df.copy()  # we should use this one really, but takes too much RAM

### 2. One-hot encode remaining values

In [12]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False, handle_unknown="error")

In [13]:
df_categoricals_onehot = enc.fit_transform(df_preprocessed[ordered_cat_columns])

In [14]:
from collections import OrderedDict
df_onehot = OrderedDict()

col_count = 0
for col, values in zip(ordered_cat_columns, enc.categories_):
    for v_idx, v in enumerate(values):
        df_onehot[f"{col}_{v}"] = df_categoricals_onehot[:, col_count + v_idx].astype(int)
    
    col_count += len(values)

# Add label and remaining numerical features
df_onehot.update({
    col: df[col].to_numpy() for col in df.columns if col not in set(ordered_cat_columns)
})

df_onehot.move_to_end(SENSITIVE_COL, last=False)
df_onehot.move_to_end(dataset_details.target, last=False)

In [15]:
df_onehot = pd.DataFrame(df_onehot)
df_onehot

,PINCP,SEX,OCCP_-1,OCCP_0,OCCP_10,OCCP_30,OCCP_41,OCCP_42,OCCP_50,OCCP_91,...,POBP_460,POBP_461,POBP_466,POBP_468,POBP_500,POBP_507,POBP_514,SCHL,AGEP,WKHP
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,18.0,18.0,21.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,17.0,53.0,40.0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,16.0,41.0,40.0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,18.0,18.0,2.0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19.0,21.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1655424,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20.0,55.0,30.0
1655425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,18.0,41.0,40.0
1655426,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,21.0,34.0,50.0
1655427,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19.0,49.0,40.0


In [16]:
train_df_onehot = df_onehot.loc[train_idx]
train_df_onehot.to_csv(data_dir / f"ACSIncome.train.onehot.header{'.all-values' if not MERGE_LOW_PREVALENCE_VALUES else ''}.csv", sep='\t', header=True)

test_df_onehot = df_onehot.loc[test_idx]
test_df_onehot.to_csv(data_dir / f"ACSIncome.test.onehot.header{'.all-values' if not MERGE_LOW_PREVALENCE_VALUES else ''}.csv", sep='\t', header=True)

if VAL_PCT > 0:
    val_df_onehot = df_onehot.loc[val_idx]
    val_df_onehot.to_csv(data_dir / f"ACSIncome.validation.onehot.header{'.all-values' if not MERGE_LOW_PREVALENCE_VALUES else ''}.csv", sep='\t', header=True)

___
___

## Write DFs to disk

In [17]:
# Persist train and test DFs
df.to_csv(data_dir / "ACSIncome.csv")

train_df.to_csv(data_dir / "ACSIncome.train.csv")
train_df.to_csv(data_dir / "ACSIncome.train.header.csv", sep='\t', header=True, index_label="index")
train_df.to_csv(data_dir / "ACSIncome.train.preprocessed-for-lightgbm-cpp.csv", sep='\t', header=False)

test_df.to_csv(data_dir / "ACSIncome.test.csv")
test_df.to_csv(data_dir / "ACSIncome.test.header.csv", sep='\t', header=True, index_label="index")
test_df.to_csv(data_dir / "ACSIncome.test.preprocessed-for-lightgbm-cpp.csv", sep='\t', header=False)

In [18]:
if VAL_PCT > 0:
    val_df.to_csv(data_dir / "ACSIncome.validation.csv")
    val_df.to_csv(data_dir / "ACSIncome.validation.header.csv", sep='\t', header=True, index_label="index")
    val_df.to_csv(data_dir / "ACSIncome.validation.preprocessed-for-lightgbm-cpp.csv", sep='\t', header=False)

## Write version for LightGBM (C++)

In [19]:
# Write column ordering to disk
cols_file_path = data_dir / "cols_order.csv"
with open(cols_file_path, "w") as out_f:
    out_f.write(",".join(["index"] + cols_order))

___
___
# Equalize prevalences among protected groups (unserample LNs)

In [20]:
delta = 1e-5
LABEL_COL = dataset_details.target

# Maximum prevalence
max_prevalence = train_df.groupby(SENSITIVE_COL).mean()[LABEL_COL].max()

train_df_eq_prev: pd.DataFrame = None
for g in train_df[SENSITIVE_COL].unique():
    group_data = train_df[train_df[SENSITIVE_COL] == g]
    
    if group_data[LABEL_COL].mean() < max_prevalence - delta:
        group_data_LP = group_data[group_data[LABEL_COL] == 1]
        group_data_LN = group_data[group_data[LABEL_COL] == 0]
        
        LPs = len(group_data_LP)
        LNs = len(group_data_LN)
        assert LPs + LNs == len(group_data)

        # Compute number of LNs to achieve target prevalence
        target_num_LN = int((LPs / max_prevalence) - LPs)

        group_data = pd.concat(
            (
                group_data_LP,
                group_data_LN.sample(n=target_num_LN),
            ),
            axis=0,
        ).sample(frac=1) # re-shuffle rows

    if train_df_eq_prev is None:
        train_df_eq_prev = group_data
    else:
        train_df_eq_prev = pd.concat(
            (train_df_eq_prev, group_data),
            axis=0)

assert all(train_df_eq_prev.groupby(SENSITIVE_COL).mean()[LABEL_COL] >= max_prevalence - delta)

In [21]:
train_df.groupby(SENSITIVE_COL).mean()[LABEL_COL]

SEX
0    0.442827
1    0.291470
Name: PINCP, dtype: float64

In [22]:
train_df_eq_prev.groupby(SENSITIVE_COL).mean()[LABEL_COL]

SEX
0    0.442827
1    0.442828
Name: PINCP, dtype: float64

In [23]:
train_df_eq_prev.to_csv(data_dir / "ACSIncome.train-equal-prev.csv")
train_df_eq_prev.to_csv(data_dir / "ACSIncome.train-equal-prev.header.csv", sep='\t', header=True, index_label="index")
train_df_eq_prev.to_csv(data_dir / "ACSIncome.train-equal-prev.preprocessed-for-lightgbm-cpp.csv", sep='\t', header=False)

___